**VERSION** LSTM Multistep v1.1 13-Nov-2019.
Prototype date: In-Progress
***
**PROBLEM FRAME**

*'Build a model that can forecast **4** values ahead'*

**ACKNOWLEDGEMENT**

*This notebook is heavily inspired by [Jason Brownlee](https://machinelearningmastery.com/how-to-develop-lstm-models-for-time-series-forecasting/)*.

*Special thanks to Anand and Howard.* 

**NOTE**

This is a back-up file, not a final version. It still cannot calculate RMSE and predict values.





**1. METHODS**

a) Split methods

In [0]:
def split_dataset(dataset, fraction = 0.7):
  ## Split the dataset into train and test set ##
  train_size = int(len(dataset) * fraction)
  test_size = len(dataset) - train_size
  train_data, test_data = dataset[0:train_size,:], dataset[train_size:len(scaled_data),:]
  return train_data, test_data

def split_sequence(sequence, n_steps_in, n_steps_out):
	## Split a univariate sequence into samples ##
  X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps_in
		out_end_ix = end_ix + n_steps_out
		# check if we are beyond the sequence
		if out_end_ix > len(sequence):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix:out_end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

b) Evaluation methods

**2. MODEL**

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

def build_model(X_train, y_train, n_steps_in=48, n_steps_out = 4, epochs=50, verbose=3, history = True): 
  ## Build an LSTM model ##
  # Define model
  model = Sequential()
  model.add(LSTM(100, activation='relu', return_sequences=True, input_shape=(n_steps_in, n_features)))
  model.add(LSTM(100, activation='relu'))
  model.add(Dense(n_steps_out))
  model.compile(optimizer='adam', loss='mse')
  # Train model
  history = model.fit(X_train, y_train, epochs=epochs, verbose=verbose)
  if(history): 
    print(history)
  return model


**3. IMPLEMENT**

a) Import the dataset 

In [0]:
from pandas import read_csv, to_datetime
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
plt.rcParams['figure.figsize'] = 20, 10

#Import the dataset
dataset = read_csv(r'UTSusage.csv', header = 0, parse_dates = [['Date', 'Time']])
dataset.Date_Time = to_datetime(dataset.Date_Time, format = '%Y/%m/%d %H.%M')
dataset.index = dataset.Date_Time

#Extract the needed column
column_needed = ['DemandKva']
dataset = dataset[column_needed]
dataset = dataset.astype('float32')

#Normalise the data
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(dataset)

In [63]:
print(dataset.head())
print(dataset.tail())
print(dataset.describe())

                      DemandKva
Date_Time                      
2019-01-01 00:00:00  131.070007
2019-01-01 00:30:00  113.879997
2019-01-01 01:00:00  109.949997
2019-01-01 01:30:00  106.930000
2019-01-01 02:00:00  124.400002
                      DemandKva
Date_Time                      
2019-04-30 21:30:00  107.550003
2019-04-30 22:00:00  109.989998
2019-04-30 22:30:00  109.419998
2019-04-30 23:00:00  109.519997
2019-04-30 23:30:00  103.089996
         DemandKva
count  5760.000000
mean    141.861221
std      37.909058
min      53.730000
25%     116.157503
50%     130.309998
75%     160.449997
max     380.269989


b) Split

In [0]:
n_steps_in= 48
n_steps_out = 4
# split dataset into train and test set
train, test = split_dataset(scaled_data)
# split into samples
X_train, y_train = split_sequence(train, n_steps_in, n_steps_out)
X_test, y_test = split_sequence(test, n_steps_in, n_steps_out)


c) Train LSTM model

In [65]:
model = build_model(X_train, y_train, n_steps_in=48, n_steps_out = 4, epochs = 100, verbose = 10)

Train on 3980 samples
Epoch 1/100
Epoch 2/100
Epoch 3/100
Epoch 4/100
Epoch 5/100
Epoch 6/100
Epoch 7/100
Epoch 8/100
Epoch 9/100
Epoch 10/100
Epoch 11/100
Epoch 12/100
Epoch 13/100
Epoch 14/100
Epoch 15/100
Epoch 16/100
Epoch 17/100
Epoch 18/100
Epoch 19/100
Epoch 20/100
Epoch 21/100
Epoch 22/100
Epoch 23/100
Epoch 24/100
Epoch 25/100
Epoch 26/100
Epoch 27/100
Epoch 28/100
Epoch 29/100
Epoch 30/100
Epoch 31/100
Epoch 32/100
Epoch 33/100
Epoch 34/100
Epoch 35/100
Epoch 36/100
Epoch 37/100
Epoch 38/100
Epoch 39/100
Epoch 40/100
Epoch 41/100
Epoch 42/100
Epoch 43/100
Epoch 44/100
Epoch 45/100
Epoch 46/100
Epoch 47/100
Epoch 48/100
Epoch 49/100
Epoch 50/100
Epoch 51/100
Epoch 52/100
Epoch 53/100
Epoch 54/100
Epoch 55/100
Epoch 56/100
Epoch 57/100
Epoch 58/100
Epoch 59/100
Epoch 60/100
Epoch 61/100
Epoch 62/100
Epoch 63/100
Epoch 64/100
Epoch 65/100
Epoch 66/100
Epoch 67/100
Epoch 68/100
Epoch 69/100
Epoch 70/100
Epoch 71/100
Epoch 72/100
Epoch 73/100
Epoch 74/100
Epoch 75/100
Epoch 76/100

c) Prediction

In [64]:
# Demonstrate
yhat = model.predict(X_test, verbose=10)
print(scaler.inverse_transform(yhat))

[[114.18548  115.56992  114.48395  116.33073 ]
 [115.55115  116.88935  115.77501  117.82671 ]
 [116.52499  117.55788  116.562584 118.39691 ]
 ...
 [112.55112  112.934166 110.60605  110.29754 ]
 [109.76127  110.34313  108.85847  109.34944 ]
 [107.45459  107.74198  107.079056 107.98145 ]]
